In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import models

In [2]:
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [3]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.alexnet(pretrained=True)

C:\Users\admin\anaconda3\envs\ai\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\admin\anaconda3\envs\ai\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\admin/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth
100.0%


In [10]:
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
model = model.to(device)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [11]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train()  # 모델을 학습 모드로 설정 (Dropout, BatchNorm 등 활성화)
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()         # 기울기 초기화
        outputs = model(inputs)        # 순전파 (forward pass)
        loss = criterion(outputs, labels)  # 손실 계산
        loss.backward()               # 역전파 (backward pass)
        optimizer.step()              # 파라미터 업데이트

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

Epoch [1/5], Loss: 1.6096
Epoch [2/5], Loss: 1.0492
Epoch [3/5], Loss: 0.8497
Epoch [4/5], Loss: 0.7577
Epoch [5/5], Loss: 0.6732


In [12]:
model.eval()  # 평가 모드로 설정 (Dropout, BatchNorm 비활성화)
correct = 0
total = 0

with torch.no_grad():  # 그래디언트 계산 비활성화 → 메모리/속도 최적화
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)  # 가장 높은 값을 갖는 클래스 선택
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')

Test Accuracy: 76.11%
